In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Convenience for making pretty plots
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 300

In [ ]:
from gpax.kernels import RBFKernel
from gpax.gp import ExactGP
from gpax.state import set_rng_key, silent_mode
from gpax.acquisition import UpperConfidenceBound
from gpax.utils.experiments import SimpleSinusoidal1d

# Basic active learning

Here we present a very simple experiment showcasing how active learning can be done using GPax. We'll use a fully exploratory acquisition function: maximum variance, which is initialized using `UpperConfidenceBound` (UCB) with `beta=float('inf')` (or `np.inf` or related). This tells UCB to only consider the posterior variance when choosing the next experiment. The long story short is that at each step, the point at which the variance is largest will be chosen for the next experiment, and Monte Carlo sampling will be used to determine where that maximum is.

In [ ]:
set_rng_key(1)

In [ ]:
experiment = SimpleSinusoidal1d()
x, y = None, None
x_grid = experiment.get_dense_coordinates(ppd=100)
acqf = UpperConfidenceBound(beta=np.inf, q=1, bounds=experiment.domain, fast=True)

In [ ]:
with silent_mode():
    for iteration in tqdm(range(20), disable=True):   
        gp = ExactGP(kernel=RBFKernel(), x=x, y=y, y_std=None, hp_samples=500, observation_noise=False)
        if x is not None:
            gp.fit()
        x_star, y_star = acqf.optimize(gp, n=1000, method="Halton")
        y_exp = experiment(x_star)
    
        mu, sd = gp.predict(x_grid)
        ci = [mu - 2*sd, mu + 2*sd]
        
        fig, axs = plt.subplots(1, 2, figsize=(6, 1), sharex=True)
        ax = axs[0]
        if x is not None:
            ax.scatter(x, y)
        ax.plot(x_grid, mu, "r-")
        ax.plot(x_grid, experiment(x_grid), "k--")
        ax.fill_between(x_grid.squeeze(), *ci, color="red", alpha=0.5, linewidth=0)
        ax.scatter(x_star, y_exp, marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Experiment")
    
        ax = axs[1]
        v = acqf(gp, x_grid)
        if x is not None:
            ax.scatter(x, acqf(gp, x))
        ax.plot(x_grid, v, "b-")
        ax.scatter(x_star, acqf(gp, x_star), marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Acquisition Function")
    
        plt.subplots_adjust(wspace=0.4)
        
        plt.show()
    
        if x is not None:
            x = np.append(x, x_star, axis=0)
            y = np.append(y, y_exp)
        else:
            x = np.atleast_2d(x_star).copy()
            y = np.atleast_1d(y_exp).copy()


Something a bit more exploitative:

In [ ]:
from gpax.utils.experiments import SimpleDecayingSinusoidal1d
from gpax.acquisition import ExpectedImprovement

In [ ]:
set_rng_key(1)

In [ ]:
experiment = SimpleDecayingSinusoidal1d()
x, y = None, None
x_grid = experiment.get_dense_coordinates(ppd=100)
acqf = ExpectedImprovement(q=1, bounds=experiment.domain, fast=True)

In [ ]:
with silent_mode():
    for iteration in tqdm(range(20), disable=True):    
        gp = ExactGP(kernel=RBFKernel(), x=x, y=y, y_std=None, hp_samples=500, observation_noise=False)
        if x is not None:
            gp.fit()
        x_star, y_star = acqf.optimize(gp, n=1000, method="Halton")
        y_exp = experiment(x_star)
    
        mu, sd = gp.predict(x_grid)
        ci = [mu - 2*sd, mu + 2*sd]
        
        fig, axs = plt.subplots(1, 2, figsize=(6, 1), sharex=True)
        ax = axs[0]
        if x is not None:
            ax.scatter(x, y)
        ax.plot(x_grid, mu, "r-")
        ax.plot(x_grid, experiment(x_grid), "k--")
        ax.fill_between(x_grid.squeeze(), *ci, color="red", alpha=0.5, linewidth=0)
        ax.scatter(x_star, y_exp, marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Experiment")
    
        ax = axs[1]
        v = acqf(gp, x_grid)
        if x is not None:
            ax.scatter(x, acqf(gp, x))
        ax.plot(x_grid, v, "b-")
        ax.scatter(x_star, acqf(gp, x_star), marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Acquisition Function")
    
        plt.subplots_adjust(wspace=0.4)
        
        plt.show()
    
        if x is not None:
            x = np.append(x, x_star, axis=0)
            y = np.append(y, y_exp)
        else:
            x = np.atleast_2d(x_star).copy()
            y = np.atleast_1d(y_exp).copy()
